In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,BatchNormalization,Dense,Dropout,Flatten,GlobalAveragePooling2D,Activation,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import cv2

In [3]:
train_path='/kaggle/input/fer2013/train'
test_path='/kaggle/input/fer2013/test'

In [4]:
os.listdir(train_path+'/angry/')[0:10]

In [5]:
image=cv2.imread(train_path+'/angry/Training_51319902.jpg')
import matplotlib.pyplot as plt
plt.imshow(image)

In [6]:
img_gen=ImageDataGenerator(rotation_range=30, fill_mode='nearest',width_shift_range=0.2, height_shift_range=0.2,
                                 horizontal_flip=True, vertical_flip=True,
                                 brightness_range=[0.4,1.5],
                                 zoom_range=0.3,
                                 rescale=1/255.0)

In [7]:
train_data=img_gen.flow_from_directory(train_path,color_mode = 'grayscale',class_mode='categorical',target_size=(48,48),shuffle=True,batch_size=32)

In [8]:
test_data=img_gen.flow_from_directory(test_path,color_mode = 'grayscale',class_mode='categorical',target_size=(48,48),shuffle=True,batch_size=512)

In [9]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = 'same', kernel_initializer="he_normal",
                 input_shape = (48, 48, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding = "same", kernel_initializer="he_normal", 
                 input_shape = (48, 48, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #2: second CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #3: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #4: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #5: first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #6: second set of FC => RELU layers
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #7: softmax classifier
model.add(Dense(7, kernel_initializer="he_normal"))
model.add(Activation("softmax"))


In [10]:
model.compile(optimizer=Adam(0.0004),loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early=EarlyStopping(monitor='val_loss',patience=4)
reduction=ReduceLROnPlateau(patience=2,monitor='val_loss',factor=0.1)

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint


In [13]:
checkpoint = ModelCheckpoint(filepath='/kaggle/output/',
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

In [14]:
model.fit(train_data,epochs=100,validation_data=test_data,callbacks=[early,reduction])

In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

nb_train_samples = 28273
nb_validation_samples = 3534

# We need to recreate our validation generator with shuffle = false
validation_generator = img_gen.flow_from_directory(
        '/kaggle/input/fer2013/test',
        color_mode = 'grayscale',
        target_size=(50, 50),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

In [ ]:
validation_generator = img_gen.flow_from_directory(
        '/kaggle/input/fer2013/test',
        color_mode = 'grayscale',
        target_size=(50, 50),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)

In [31]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import re
import matplotlib.pyplot as plt

def draw_test(name, pred, im, true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 160, 0, 0, 300 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, "predited - "+ pred, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.putText(expanded_image, "true - "+ true_label, (20, 120) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    cv2.imshow(name,expanded_image)


def getRandomImage(path, img_width, img_height):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width, img_height),grayscale=True), final_path, path_class

# dimensions of our images
img_width, img_height = 48, 48

# We use a very small learning rate 
#model.compile(loss = 'categorical_crossentropy',
 
    #optimizer = RMSprop(lr = 0.001),
     #         metrics = ['accuracy'])

files = []
predictions = []
true_labels = []

#fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 12))
#plt.subplots_adjust(hspace=0.5)
#fig.suptitle("Model Predictions", fontsize=18, y=0.95)
# predicting images
for i in range(0, 10):
    path = '/kaggle/input/fer2013/test/' 
    img, final_path, true_label = getRandomImage(path, img_width, img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x * 1./255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size = 10)
    predictions.append(classes)
    
for i in range(0, len(files)):
    
    #print(np.argmax(predictions[i][0]))
    #print(class_labels[np.argmax(predictions[i][0])])
    pred="predited - "+ class_labels[np.argmax(predictions[i][0])]
    true="true - "+ true_labels[i]
    image = cv2.imread((files[i]))
    image = cv2.resize(image, (400,400), fx=3, fy=3, interpolation = cv2.INTER_CUBIC)
    cv2.putText(image, pred, (10, 40) , cv2.FONT_HERSHEY_SIMPLEX,1.2, (255,0,25), 3)
    cv2.putText(image, true, (10, 90) , cv2.FONT_HERSHEY_SIMPLEX,1.2, (0,0,250), 3)
    plt.imshow(image)
    plt.show()
    #draw_test("Prediction", pred, image, true)
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
#data=pd.read_json('/kaggle/input/news-category-dataset/News_Category_Dataset_v2.json')